In [29]:
import numpy as np
import pandas as pd
import rasterio

In [36]:
travel_time = r"C:\Proposals\CCA\LPG_CODE_TEST_LAYERS\travel_time.tif"

In [52]:
def LPG_transportation_cost(travel_time):
    
    """"The cost of transporting LPG. See https://iopscience.iop.org/article/10.1088/1748-9326/6/3/034002/pdf for the formula 
    
    (2 * diesel consumption per h * national diesel price * travel time)/transported LPG
    
    Each truck is assumed to transport 2,000 kg LPG 
    (3.5 MT truck https://www.wlpga.org/wp-content/uploads/2019/09/2019-Guide-to-Good-Industry-Practices-for-LPG-Cylinders-in-the-Distribution-Channel.pdf)
    National diesel price in Nepal is assumed to be 0.88 USD/l
    Diesel consumption per h is assumed to be 14 l/h 
    (https://www.iea.org/reports/fuel-consumption-of-cars-and-vans)
    
    Based on https://iopscience.iop.org/article/10.1088/1748-9326/aa6fd0/meta#erlaa6fd0fd2 125 kg LPG is needed for cooking
    per year if LPG is used exclusively 
    
    Parameters
    ----------
    arg1 : national_LPG_price
        The national LPG price (USD)
    arg2 : travel_time_raster
        Hour to travel between each point and the startpoints as array
    Returns
    ----------
    The cost of LPG in each cell
    
    """
    with rasterio.open(travel_time) as src:
        trav = src.read(1)
        
    return ((2*14*0.88*trav)/2000)

LPG_transportation_cost = LPG_transportation_cost(travel_time)
print(np.nanmax(LPG_transportation_cost))

1.8472780494737795


* I need 125 kg LPG/year
* Cost of LPG 1400 per cylinder NRs
* Each cylinder includes 15 kg
* Cost per kg is 98.6 NRs
* 1 NR = 0.014 USD
* Cost per kg is 1.38 USD
* Total cost is 125*1.38 if LPG is used exclusively 

* Price changes with travel time Szabo (2*12*national_LPG_price*traveltime/300) <-- asuming that we are transporting 300  


In [35]:
def LPG_fuel_cost_calculator(LPG_price,
                             LPG_truck_volume,
                             traveltime,
                             LHV_LPG,
                             efficiency):
    
    """We apply the Szabo formula to calculate the transport cost for the LPG
    Formulae is::
        p = (p_lpg + 2*p_lpg*consumption*time/volume)*(1/mu)*(1/LHVlpg)
    Arguments
    ---------
    LPG_price: float
    LPG_truck_consumption: float
    LPG_truck_volume: float
    traveltime: raster (numpy array)
    efficiency: float
    Returns
    -------
    numpy.ndarray
    """
    return (LPG_price + 2 * LPG_price * LPG_truck_consumption *
            traveltime / LPG_truck_volume) / LHV_LPG / efficiency


with rasterio.open(travel_time) as src:
    fric = src.read(1)

149.94140011962494